In [118]:
# Startup

# FBMP Scraper
from splinter import Browser
from bs4 import BeautifulSoup as soup
import re
import pandas as pd
import matplotlib.pyplot as plt
import time
from datetime import date

# Discord
import os
import discord

# Set up search parameters ***leave as '' for no option***
csv_file_path = r'D:\iso\xp\Shared Folder\cars.csv'
city = "calgary"
base_url = f"https://www.facebook.com/marketplace/{city}/vehicles?"
min_price = 1235
max_price = 16000
min_mileage = 40000
max_mileage = 180000
min_year = 2008
max_year = 2014
transmission = '' # 'automatic' or 'manual'
make = 'audi' 
model = 'a4'

# Set up full url
url = f"{base_url}minPrice={min_price}&maxPrice={max_price}&maxMileage={max_mileage}&maxYear={max_year}&minMileage={min_mileage}&minYear={min_year}" 
if transmission != '':
    url = url + f'&transmissionType={transmission}'
if make != '':
    url = url + f'&make={make}'
    if model != '':
        url = url + f'&model={model}'
url = url + '&sortBy=creation_time_descend&exact=false)'

# Create function
def skim():
    browser = Browser('chrome') # Set up Splinter
    browser.visit(url) # Visit the website
    
    # Scroll down to load more results
    scroll_count = 0 # ***Define the number of times to scroll the page***
    scroll_delay = 2
    for _ in range(scroll_count): 
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);") # Execute JavaScript to scroll to the bottom of the page
        time.sleep(scroll_delay) # Pause for a moment to allow the content to load
    html = browser.html
    market_soup = soup(html, 'html.parser')
    # Check if HTML was scraped correctly (uncomment below):
    # print(market_soup)
    time.sleep(4)
    browser.quit()
    # Extract all the necessary info and insert into lists
    titles_div = market_soup.find_all('span', class_="x1lliihq x6ikm8r x10wlt62 x1n2onr6")
    titles_list = [title.text.strip() for title in titles_div]
    prices_div = market_soup.find_all('span', class_="x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x xudqn12 x676frb x1lkfr7t x1lbecb7 x1s688f xzsf02u")
    prices_list = [price.text.strip() for price in prices_div]
    mileage_div = market_soup.find_all('span', class_="x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x x4zkp8e x3x7a5m x1nxh6w3 x1sibtaa xo1l8bm xi81zsa")
    mileage_list = [mileage.text.strip() for mileage in mileage_div]
    urls_div = market_soup.find_all('a', class_="x1i10hfl xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz x1heor9g x1lku1pv")
    urls_list = [url.get('href') for url in urls_div]
    # Make sure the lengths all match
    print(len(titles_list))
    print(len(prices_list))
    print(len(mileage_list) / 2)
    print(len(urls_list))
    
    # Data formatting
    pattern = re.compile(r'(\w+(?:-\w+)?, [A-Z]{2})') # Create a regular expression pattern to match city and state entries like "City, State"
    mileage_list2 = []
    for item in mileage_list: # Iterate through the original mileage entries
        mileage_list2.append(item)
        if pattern.match(item) and len(mileage_list2) >= 2 and pattern.match(mileage_list2[-2]): # Check if the current mileage entry matches the pattern and there are at least two entries in the adjusted list
            mileage_list2.insert(-1, '0K km') # If the conditions are met, insert "0K km" in between the two consecutive city and state entries
    # Extract mileage from mileage list
    # Define regular expressions to extract numeric mileage values in "K km" and "K miles" format
    mileage_pattern_km = r'(\d+)K km'
    mileage_pattern_miles = r'(\d+)K miles'
    mileage_clean = []
    for item in mileage_list2:
        match_mileage_km = re.search(mileage_pattern_km, item) # Try to find a match for the "K km" format
        match_mileage_miles = re.search(mileage_pattern_miles, item) # Try to find a match for the "K miles" format
        if match_mileage_km or match_mileage_miles: # Check if either of the formats is found
            if match_mileage_km: # If "K km" format is found, convert it to meters and append to the cleaned list
                mileage_clean.append(int(match_mileage_km.group(1)) * 1000)
            else:
                mileage_clean.append(int(match_mileage_miles.group(1)) * 1600) # If "K miles" format is found, convert it to meters and append to the cleaned list
                
    # Add all values to a list of dictionaries
    vehicles_list = []
    for i, item in enumerate(titles_list):
        cars_dict = {}
    
        title_split = titles_list[i].split()
    
        cars_dict["Year"] = int(title_split[0])
        cars_dict["Make"] = title_split[1]
        cars_dict["Model"] = title_split[2]
        cars_dict["Price"] = int(re.sub(r'[^\d.]', '', prices_list[i]))
        cars_dict["Mileage"] = mileage_clean[i]
        head, sep, tail = urls_list[i].partition('/?ref') # Shortens URL for comparison
        cars_dict["URL"] = head
        cars_dict["Date"] = f'{date.today()}'
        cars_dict["Time"] = f'{time.strftime("%H:%M", time.localtime())}'
        vehicles_list.append(cars_dict)
    return vehicles_list
url

'https://www.facebook.com/marketplace/calgary/vehicles?minPrice=1235&maxPrice=16000&maxMileage=180000&maxYear=2014&minMileage=40000&minYear=2008&make=audi&model=a4&sortBy=creation_time_descend&exact=false)'

In [119]:
# Wipe data file
totalDataFile = [{'Year': '#null', 'Make': '#null', 'Model': '#null', 'Price': '#null', 'Mileage': '#null', 'URL': '#null', 'Date and Time': '#null'}] # Null values to start loop, removed after first car is listed

In [121]:
# Collect data once
recentDataFile = skim() # Execute skimming function

tempDataFile = [] # Creates temp list to avoid infinite loop
for i, car in enumerate(totalDataFile): # Creates copy of totalDataFile variable
    tempDataFile.append(totalDataFile[i])
for i, listing in enumerate(recentDataFile): # Checks for duplicate listings using the URL
    true = 0
    for j, car in enumerate(totalDataFile):
        if recentDataFile[i].get('URL') == totalDataFile[j].get('URL') and true == 0:
            true = 1
        if j+1 == len(totalDataFile) and true == 0:
            tempDataFile.append(listing)
            print('New Car Listed')
            if tempDataFile[0].get('Year') == '#null': # Remove null row if this is first run
                tempDataFile.pop(0)
                
totalDataFile = tempDataFile # Removes need of temporaryDataFile
vehicles_df = pd.DataFrame(totalDataFile) # Creates a chart
pd.set_option('display.max_colwidth', None)
vehicles_df['URL'] = 'https://www.facebook.com/' + vehicles_df['URL'] # Convert to proper URLs
vehicles_df.to_csv(csv_file_path, index=False)# Convert and save recent scan to csv

20
20
20.0
20
